# rectification

In [ ]:
import glob
import json
import os
from multiprocessing import Pool, pool

import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# load the parameters
params = json.load(open('/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/stereo_parameters.json'))

cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
               params['CameraParameters1']['TangentialDistortion'] + \
               [params['CameraParameters1']['RadialDistortion'][2]]
distCoeffs1 = np.array(distCoeffs1)

distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
               params['CameraParameters2']['TangentialDistortion'] + \
               [params['CameraParameters2']['RadialDistortion'][2]]
distCoeffs2 = np.array(distCoeffs2)

R = np.array(params['RotationOfCamera2']).transpose()
T = np.array(params['TranslationOfCamera2']).transpose()

In [ ]:
cv2.__version__

In [ ]:
imageSize = (4096, 3000)

In [ ]:
# rectification
(R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, imageSize, R, T, None, None, None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)

left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)

### ALL FOLDERS

In [ ]:
base_dir = '/root/data/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/'
experiences = os.listdir(base_dir)
all_image_path = []
for experience in experiences:
    folder_path = os.path.join(base_dir, experience)
    if not os.path.isdir(folder_path):
        continue
    if "rectified" in folder_path:
        continue
    if "reference" in folder_path:
        continue
    print(folder_path)
    all_image_path += glob.glob(folder_path + '/*.jpg')

In [ ]:
all_image_path[0]

In [ ]:
# left_paths = sorted(glob.glob('/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001/left*'))
# right_paths = sorted(glob.glob('/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001/right*'))

In [ ]:
def remap_and_save(img_path):
    exp_number = img_path.split('/')[-2]
    img = cv2.imread(img_path)
    if 'left' in img_path:
        img_remap = cv2.remap(img, left_maps[0], left_maps[1], cv2.INTER_LANCZOS4)
    else:
        img_remap = cv2.remap(img, right_maps[0], right_maps[1], cv2.INTER_LANCZOS4)
    new_path = img_path.replace(exp_number, exp_number + '_rectified')
    print(new_path)
    if not os.path.isdir(os.path.dirname(new_path)):
        os.makedirs(os.path.dirname(new_path))
    cv2.imwrite(new_path, img_remap)

In [ ]:
# multiproceesing
p = Pool(6)

In [ ]:
p.map(remap_and_save, all_image_path)

# display

In [ ]:
assert False, "Oh no! This assertion failed!"